<a href="https://colab.research.google.com/github/dimitrod/ehu_nlp_dimathina/blob/Evaluation/evaluate_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model Evaluation
In this script you can evaluate any model that has been developed during the project using the evaluation data.

By default the evaluation data has been set to be the first 7900 questions from the train split of the triviaqa dataset.
If you want to change the evaluation data to be a different split you can do that using the *load_evaluation_data.py* script in the utils package. A detailed guide for this is in the *evaluation_data.ipynb* script in the main project folder of this branch

In this branch we used a dummy model to check if the evaluation chain works. In further branches this model will be swapped for the developed model. The dummmy model always gives the correct answer to any question.
If you want to interact with the model directly you can do this in the *test_model.ipynb* script in the main project folder.



---



## Setup
To setup the evaluation chain run to the following steps

Clone Current branch from github

In [1]:
!git clone --branch Evaluation https://github.com/dimitrod/ehu_nlp_dimathina.git

Cloning into 'ehu_nlp_dimathina'...
remote: Enumerating objects: 144, done.
remote: Counting objects: 100% (144/144), done.
remote: Compressing objects: 100% (139/139), done.
remote: Total 144 (delta 77), reused 3 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (144/144), 2.52 MiB | 4.07 MiB/s, done.
Resolving deltas: 100% (77/77), done.


Navigate into the directory and clone the triviaqa evaluation repo

In [2]:
%cd ehu_nlp_dimathina/
!git clone https://github.com/mandarjoshi90/triviaqa.git

/content/ehu_nlp_dimathina
Cloning into 'triviaqa'...
remote: Enumerating objects: 70, done.
remote: Total 70 (delta 0), reused 0 (delta 0), pack-reused 70 (from 1)
Receiving objects: 100% (70/70), 20.60 KiB | 527.00 KiB/s, done.
Resolving deltas: 100% (28/28), done.


Navigate into the triviaqa directory and install the requirements

In [3]:
%cd triviaqa/
!pip install -r requirements.txt

/content/ehu_nlp_dimathina/triviaqa


Navigate into the Evaluation directory and install the requirements

In [4]:
%cd ../Evaluation
!pip install -r requirements.txt

/content/ehu_nlp_dimathina/Evaluation
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


Navigate back into the repo and set the working directory to be the current directory

In [5]:
%cd ..
import os

os.chdir(os.path.abspath("."))

/content/ehu_nlp_dimathina




---



## Executing the Evaluation Chain
Now you can run the Evaluation chain with the model of choice. In this case it is a dummy model to test the functionality of the evaluation chain. The dummy model always looksup the correct answer to every question.

To start the evaluation chain type the following command

`python3 -m Evaluation.evaluationchain --model_directory "Dummy_Model" --model_module "dummy_model"--model_class "dummy_model" --split_size 7900`

If you don't want to test the model against the entire evaluation set you can choose a smaller split size. Otherwise the maximum value of the split size is 7900

In [6]:
!python3 -m Evaluation.evaluation_chain --model_directory "Dummy_Model" --model_module "dummy_model" --model_class "dummy_model" --split_size 7900

Loading Questions: 100% 7900/7900 [00:00<00:00, 942647.48it/s]
Loading model...
Model initiated
Starting QA...
Retrieving Answers: 100% 7900/7900 [00:00<00:00, 838945.76it/s]
QA complete
Starting evaluation...

Evaluation complete
Saving results...
Test results:
{'exact_match': 100.0, 'f1': 100.0, 'common': 7900, 'denominator': 7900, 'pred_len': 7900, 'gold_len': 7900}



The results of the test are saved in the file *model_name*_split_size=*split_size*_results.txt in the Evaluation package